In [9]:
from tqdm import tqdm
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("baiges/CatGPT")
model = AutoModelForCausalLM.from_pretrained("baiges/CatGPT")

# Define the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load the Hellaswag dataset translated to Catalan
with open("../data/ca_hellaswag.json", 'r') as f:
    dataset = json.load(f)

# Define the evaluation function
def evaluate_model(model, tokenizer, dataset, device):
    total_correct = 0
    total_items = 0

    # Iterate over each entry in the dataset
    for i, item in tqdm(enumerate(dataset)):
        context = item['ctx']  # Get the context
        endings = item['endings']  # Get the possible endings
        correct_ending_idx = int(item['label'])  # Index of the correct ending

        # Tokenize the context and move it to the device
        context_input = tokenizer(context, return_tensors="pt").input_ids.to(device)

        avg_loss_values = []
        for ending in endings:
            # Tokenize the ending and move it to the device
            ending_input = tokenizer(ending, return_tensors="pt").input_ids.to(device)
            # Concatenate the context with each ending
            input_ids = torch.cat([context_input, ending_input[:, 1:]], dim=-1).to(device)

            # Calculate the loss only over the ending part
            with torch.no_grad():
                outputs = model(input_ids)
                logits = outputs.logits[:, -ending_input.size(1):, :]  # Get logits corresponding to the ending
                loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
                shift_logits = logits[:, :-1, :].contiguous()
                shift_labels = ending_input[:, 1:].contiguous()
                loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
                avg_loss = loss.mean().item()  # Calculate the average loss per token
                avg_loss_values.append(avg_loss)

        # Find the ending with the lowest average loss per token
        predicted_idx = torch.argmin(torch.tensor(avg_loss_values)).item()

        # Check if the prediction is correct and update the counters
        is_correct = predicted_idx == correct_ending_idx
        if is_correct:
            total_correct += 1
        total_items += 1

        # Calculate the accuracy for this specific item
        item_accuracy = total_correct / total_items

        # Print context, prediction, correctness, and accuracy for this item
        print(f"Context: {context}")
        print(f"Predicted Ending: {endings[predicted_idx]}")
        print(f"Correct Ending: {endings[correct_ending_idx]}")
        print(f"Is Correct: {'Yes' if is_correct else 'No'}")
        print(f"Accuracy so far: {item_accuracy:.4f}\n")

    # Calculate the overall accuracy
    accuracy = total_correct / total_items
    return accuracy

# Perform the evaluation
accuracy = evaluate_model(model, tokenizer, dataset, device)
print(f"Final Accuracy: {accuracy:.4f}")

1it [00:00,  1.59it/s]

Context: [header] Com actualitzar el teu armari de moda amb llaços [title] Busca blusons que tinguin un llaç en algun lloc de forma natural. [step] Aquesta és l'opció més fàcil per als tops, ja que no has de preocupar-te per crear un llaç que coincideixi amb el color i la mida del teu top. [title] Considera una blusa amb un estampat de llaços.
Predicted Ending: [step] En lloc d'un llaç real, una blusa amb un estampat de llaços té un patró de llaços imprès. Això introdueix el llaç al teu armari d'una manera unidimensional.
Correct Ending: [step] En lloc d'un llaç real, una blusa amb un estampat de llaços té un patró de llaços imprès. Això introdueix el llaç al teu armari d'una manera unidimensional.
Is Correct: Yes
Accuracy so far: 1.0000



2it [00:01,  1.89it/s]

Context: [header] Com evitar que el teu fill o filla esgranqui les dents [title] Entén que l'estrès pot causar bruxisme. [step] Esgranquejar les dents pot ser un signe que el teu fill o filla està estressat o ansios sobre alguna cosa. Ajudar el teu fill o filla a relaxar-se pot ser la clau per aconseguir que deixi de rascar les dents.
Predicted Ending: [title] Crea una rutina tranquil·la abans d'anar a dormir per al teu fill o filla. [step] Com s'ha esmentat en el pas anterior, esgranquejar les dents pot ser un signe que el teu fill o filla està estressat.
Correct Ending: [title] Crea una rutina tranquil·la abans d'anar a dormir per al teu fill o filla. [step] Com s'ha esmentat en el pas anterior, esgranquejar les dents pot ser un signe que el teu fill o filla està estressat.
Is Correct: Yes
Accuracy so far: 1.0000



3it [00:01,  2.12it/s]

...

23it [00:08,  2.82it/s]

Context: [header] Com portar pantalons de cigarreta [title] Escolliu una parella que caigui just sobre el turmell. [step] L'element destacat dels pantalons de cigarreta és la zona inferior de la vora. Assegureu-vos que obteniu una parella de pantalons que tingui la longitud adequada.
Predicted Ending: Podeu trobar pantalons de cigarreta a la majoria de botigues. [substeps] La zona inferior de la vora és la zona que s'enganxa fortament quan es fa foc.
Correct Ending: La part inferior de cada cama de pantaló hauria de caure just al damunt del turmell, ni massa llarg ni massa curt. [substeps] Si els vostres pantalons de cigarreta són massa llargs però per la resta s'ajusten bé, proveu d'ajustar-ne la vora.
Is Correct: No
Accuracy so far: 0.3913



24it [00:08,  2.68it/s]

Context: [header] Com guanyar recompenses de combustible [title] Investigueu programes de recompenses de combustible. [step] Moltes gasolineres diferents, supermercats i companyies de targetes de crèdit tenen programes de recompenses de combustible. Cerqueu programes a través d'internet a la vostra zona.
Predicted Ending: Cerqueu programes que siguin més convenients per a vosaltres, com ara un a través del supermercat o la gasolinera que visiteu més sovint, o un programa a través d'una targeta de crèdit que ja tingueu. [substeps] Llegiu atentament les condicions per saber com s'obtenen les recompenses, quin és el descompte màxim, quan caduquen les recompenses, i així successivament.
Correct Ending: Cerqueu programes que siguin més convenients per a vosaltres, com ara un a través del supermercat o la gasolinera que visiteu més sovint, o un programa a través d'una targeta de crèdit que ja tingueu. [substeps] Llegiu atentament les condicions per saber com s'obtenen les recompenses, quin é

KeyboardInterrupt: 